### Jingzhi Zhou

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler

In [2]:
train_df = pd.read_csv (r'C:\Users\11638\OneDrive\529\Exam2\train.csv')
test_df = pd.read_csv (r'C:\Users\11638\OneDrive\529\Exam2\test.csv')

In [3]:
train_df[:5]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
(train_df).count()

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            714
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          204
Embarked       889
dtype: int64

### a.

In [5]:
(train_df['Parch']== 0).sum()

678

In [6]:
((train_df['Parch']== 0).sum()) / ((train_df['Parch']).count())

0.7609427609427609

More than 75% passengers did not travel with parents or children: True.

In [7]:
(train_df['SibSp']!= 0).sum()

283

In [8]:
((train_df['SibSp']!= 0).sum()) / ((train_df['SibSp']).count())

0.3176206509539843

30 to 33% of passengers had siblings and/or spouse aboard: True.

In [9]:
(train_df['Fare']> 500).sum()

3

In [10]:
((train_df['Fare']> 500).sum()) / ((train_df['Fare']).count())

0.003367003367003367

Less than 1% of passengers paid a fare as high as 500 dollars: True.

In [11]:
(train_df['Age']> 65).sum()

8

In [12]:
((train_df['Age']> 65).sum()) / ((train_df['Age']).count())

0.011204481792717087

Less than 1% of passengers are 65+ years old: False.

### b.

In [13]:
# Drop columns PassengerID, Name, Ticket, Cabin.

In [14]:
train_df1 = pd.DataFrame(train_df)

In [15]:
train_df1 = train_df1.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

In [16]:
# Fill NAs values in Embark with the most common category.

In [17]:
train_df1['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

Most common category for Embarked is S. 

In [18]:
train_df1['Embarked'] = train_df1['Embarked'].fillna("S")

In [19]:
train_df1['Embarked'].value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

In [20]:
# Fill NAs values in Fare with the median value in that column.

In [21]:
train_df1["Fare"].median()

14.4542

In [22]:
train_df1['Fare'] = train_df1['Fare'].fillna("14.4542")

In [23]:
# Fill NAs values in Age with the median value in each of Pclass x Gender combination.

In [24]:
train_df1.groupby(['Pclass','Sex'], as_index=False)['Age'].median()

,Pclass,Sex,Age
0,1,female,35.0
1,1,male,40.0
2,2,female,28.0
3,2,male,30.0
4,3,female,21.5
5,3,male,25.0


In [25]:
train_df1["Age"] = train_df1.groupby(['Pclass','Sex'])["Age"].transform(lambda x: x.fillna(x.median()))

In [26]:
(train_df1).count()

Survived    891
Pclass      891
Sex         891
Age         891
SibSp       891
Parch       891
Fare        891
Embarked    891
dtype: int64

### c.

Split column Age into 5 intervals (0, 16, 32, 48, 64, 100)

In [27]:
train_df1['Age'] = pd.cut(train_df1.Age, bins=[0, 16, 32, 48, 64, 100])

In [28]:
train_df1['Age'].dtype.name

'category'

Split column Fare into 4 intervals (0, 7.9, 14.5, 31, 600)

In [29]:
train_df1['Fare'] = pd.cut(train_df1.Fare, bins=[0, 7.9, 14.5, 31, 600])

In [30]:
train_df1['Fare'].dtype.name

'category'

Create column Size by adding values from columns SibSp, Parch, then drop them keeping
the created column.

In [31]:
train_df1['Size'] =  train_df1[['SibSp', 'Parch']].sum(axis=1)

In [32]:
train_df1 = train_df1.drop(['SibSp','Parch'], 1)

Create binary column Alone with value 1 (if passenger travels alone) and 0 otherwise.

In [33]:
train_df1['Alone'] = train_df1.Size.apply(lambda x: 1 if x == 0 else 0)

Use get_dummies to convert categorical to binary columns.

In [34]:
train_df1.dtypes

Survived       int64
Pclass         int64
Sex           object
Age         category
Fare        category
Embarked      object
Size           int64
Alone          int64
dtype: object

In [35]:
train_df1 = pd.get_dummies(train_df1,columns = ['Sex','Age','Fare','Embarked'],
                   drop_first=True)

In [36]:
train_df1

,Survived,Pclass,Size,Alone,Sex_male,"Age_(16, 32]","Age_(32, 48]","Age_(48, 64]","Age_(64, 100]","Fare_(7.9, 14.5]","Fare_(14.5, 31.0]","Fare_(31.0, 600.0]",Embarked_Q,Embarked_S
0,0,3,1,0,1,1,0,0,0,0,0,0,0,1
1,1,1,1,0,0,0,1,0,0,0,0,1,0,0
2,1,3,0,1,0,1,0,0,0,1,0,0,0,1
3,1,1,1,0,0,0,1,0,0,0,0,1,0,1
4,0,3,0,1,1,0,1,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,0,1,1,1,0,0,0,1,0,0,0,1
887,1,1,0,1,0,1,0,0,0,0,1,0,0,1
888,0,3,3,0,0,1,0,0,0,0,1,0,0,1
889,1,1,0,1,1,1,0,0,0,0,1,0,0,0


Repeat steps on test.csv

In [37]:
test_df['Age'] = pd.cut(test_df.Age, bins=[0, 16, 32, 48, 64, 100])

In [38]:
test_df['Fare'] = pd.cut(test_df.Fare, bins=[0, 7.9, 14.5, 31, 600])

In [39]:
test_df['Size'] =  test_df[['SibSp', 'Parch']].sum(axis=1)

In [40]:
test_df = test_df.drop(['SibSp','Parch'], 1)

In [41]:
test_df['Alone'] = test_df.Size.apply(lambda x: 1 if x == 0 else 0)

In [42]:
test_df = pd.get_dummies(test_df,columns = ['Sex','Age','Fare','Embarked'],
                   drop_first=True)

In [43]:
test_df

,PassengerId,Pclass,Name,Ticket,Cabin,Size,Alone,Sex_male,"Age_(16, 32]","Age_(32, 48]","Age_(48, 64]","Age_(64, 100]","Fare_(7.9, 14.5]","Fare_(14.5, 31.0]","Fare_(31.0, 600.0]",Embarked_Q,Embarked_S
0,892,3,"Kelly, Mr. James",330911,NaN,0,1,1,0,1,0,0,0,0,0,1,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",363272,NaN,1,0,0,0,1,0,0,0,0,0,0,1
2,894,2,"Myles, Mr. Thomas Francis",240276,NaN,0,1,1,0,0,1,0,1,0,0,1,0
3,895,3,"Wirz, Mr. Albert",315154,NaN,0,1,1,1,0,0,0,1,0,0,0,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",3101298,NaN,2,0,0,1,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",A.5. 3236,NaN,0,1,1,0,0,0,0,1,0,0,0,1
414,1306,1,"Oliva y Ocana, Dona. Fermina",PC 17758,C105,0,1,0,0,1,0,0,0,0,1,0,0
415,1307,3,"Saether, Mr. Simon Sivertsen",SOTON/O.Q. 3101262,NaN,0,1,1,0,1,0,0,0,0,0,0,1
416,1308,3,"Ware, Mr. Frederick",359309,NaN,0,1,1,0,0,0,0,1,0,0,0,1


### d.

In [44]:
y = train_df1.Survived
X = train_df1.drop(['Survived'],axis=1)

In [45]:
X_train,X_test,y_train,y_test = train_test_split(X,y,
                                                 random_state=0)

KNN Model:

In [46]:
knn = KNeighborsClassifier()

In [47]:
param_grid = {'n_neighbors': np.arange(1, 25)}

In [48]:
knn_gscv = GridSearchCV(knn, param_grid, cv=5)

In [49]:
knn_gscv.fit(X_train, y_train);

In [50]:
knn_gscv.best_estimator_

KNeighborsClassifier(n_neighbors=19)

In [51]:
knn_best = KNeighborsClassifier(n_neighbors = 19)

In [52]:
knn_best.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=19)

In [53]:
knn_best.predict(X_test)[:5]

array([0, 0, 0, 1, 1], dtype=int64)

In [54]:
knn_best.score(X_test, y_test)

0.7847533632286996

In [55]:
# GridSearchCV approach suggests k=19 neighbor gives the best test accuracy rate, which is 0.785.

In [56]:
training_accuracy = []
test_accuracy = []

In [57]:
nn = range(1,25)

In [58]:
for k in nn:
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train, y_train)
    training_accuracy.append(model.score(X_train, 
                                         y_train))
    test_accuracy.append(model.score(X_test, 
                                     y_test))

In [59]:
print([round(i,3) for i in training_accuracy])

[0.731, 0.763, 0.786, 0.769, 0.781, 0.823, 0.819, 0.801, 0.813, 0.801, 0.796, 0.787, 0.793, 0.795, 0.795, 0.775, 0.786, 0.769, 0.792, 0.781, 0.784, 0.799, 0.79, 0.787]


In [60]:
print([round(i,3) for i in test_accuracy])

[0.65, 0.722, 0.744, 0.74, 0.731, 0.807, 0.812, 0.785, 0.798, 0.785, 0.785, 0.767, 0.767, 0.776, 0.771, 0.767, 0.776, 0.776, 0.785, 0.789, 0.78, 0.785, 0.78, 0.785]


In [61]:
max(test_accuracy)

0.8116591928251121

In [62]:
# Validation approach suggests k=7 neighbor gives the best test accuracy rate, which is 0.81166.

Support Vector Classifier:

In [63]:
param_grid= {'C':[0.01,0.1,1,10],'gamma':[0.5,1,2,3]}

In [64]:
clf = GridSearchCV(SVC(kernel='rbf'),param_grid,cv=5,
                  scoring='accuracy')
clf.fit(X_train,y_train)
clf.best_params_

{'C': 1, 'gamma': 0.5}

In [65]:
clf.best_estimator_

SVC(C=1, gamma=0.5)

In [66]:
y_pred = clf.best_estimator_.predict(X_test)
pd.DataFrame(confusion_matrix(y_test,y_pred),index=clf.best_estimator_.classes_,
                                          columns=clf.best_estimator_.classes_)

,0,1
0,123,16
1,24,60


In [67]:
clf.best_estimator_.score(X_test,y_test)

0.820627802690583

In [68]:
# The test accuracy rate using Support Vector Classifier is 0.82063.

Logistic Regression:

In [69]:
LR = LogisticRegression(solver="lbfgs", random_state=0)
LR.fit(X, y)
yhat = LR.predict(X)
pd.crosstab(y,yhat, rownames=['y'],colnames=['yhat'])

yhat,0,1
y,,
0,477,72
1,101,241


In [70]:
(477+241)/(477+72+101+241)

0.8058361391694725

In [71]:
# The test accuracy rate using Logistic Regression is 0.80584.

Random Forest:

In [72]:
parameters = {'max_depth':range(3,20),'max_features':range(2,30)}

In [73]:
RF_grid = GridSearchCV(RandomForestClassifier(random_state = 0), parameters,
                    scoring= 'neg_mean_squared_error',                    
                    cv=5, n_jobs=4,
                    return_train_score = False)

In [74]:
RF_grid.fit(X_train, y_train);

In [75]:
RF_grid.best_estimator_

RandomForestClassifier(max_depth=3, max_features=6, random_state=0)

In [76]:
random_forest_best = RandomForestClassifier(max_features=6,
                                      max_depth=3,
                                      n_estimators = 500,
                                      random_state=0)
random_forest_best.fit(X_train,y_train);

In [77]:
random_forest_best.score(X_test,y_test)

0.8071748878923767

In [78]:
# The test accuracy rate using Random Forest is 0.80717. 

Gradient Boosting:

In [79]:
parameters = {'max_depth':range(2,20),'learning_rate':[0.01,0.1,0.2,0.3,0.4,0.5,1]}

In [80]:
GB_grid = GridSearchCV(GradientBoostingClassifier(random_state = 0), parameters,
                    scoring= 'accuracy',                    
                    cv=5, n_jobs=4,
                    return_train_score = False)

In [81]:
GB_grid.fit(X_train, y_train);

In [82]:
GB_grid.best_estimator_

GradientBoostingClassifier(max_depth=2, random_state=0)

In [83]:
boosted = GradientBoostingClassifier(n_estimators = 25, learning_rate = 0.5,
                                     max_depth = 2, random_state =0)
boosted.fit(X_train,y_train)
boosted.score(X_test,y_test)

0.8295964125560538

In [84]:
# The test accuracy rate using Gradient Boosting is 0.8296. Gradient boosting gives me the highest accuracy rate so far.

Try to improve using polynomial approach on gradient boosting

In [85]:
scaler = MinMaxScaler()
poly = PolynomialFeatures()

In [86]:
Xp = poly.fit_transform(X)
Xp = pd.DataFrame(Xp,columns=poly.get_feature_names(X.columns))

In [87]:
scaler.fit(Xp);

In [88]:
Xp2 = scaler.transform(Xp)

In [89]:
Xp2 = pd.DataFrame(Xp2,columns = Xp.columns)
Xp2= Xp2.drop(['1'],axis=1)
Xp2[:5]

,Pclass,Size,Alone,Sex_male,"Age_(16, 32]","Age_(32, 48]","Age_(48, 64]","Age_(64, 100]","Fare_(7.9, 14.5]","Fare_(14.5, 31.0]",...,"Fare_(14.5, 31.0]^2","Fare_(14.5, 31.0] Fare_(31.0, 600.0]","Fare_(14.5, 31.0] Embarked_Q","Fare_(14.5, 31.0] Embarked_S","Fare_(31.0, 600.0]^2","Fare_(31.0, 600.0] Embarked_Q","Fare_(31.0, 600.0] Embarked_S",Embarked_Q^2,Embarked_Q Embarked_S,Embarked_S^2
0,1.0,0.1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [90]:
Xp_train,Xp_test,yp_train,yp_test = train_test_split(Xp2,y,
                                                 random_state=0)

In [91]:
GB_grid.fit(Xp_train, yp_train);
GB_grid.score(Xp_test,yp_test)

0.8116591928251121

In [92]:
# polynomial transformation does not improve the prediction accuracy rate on the gradient boosting, 
# so I will stick with the basic gradient boosting approach when predicting the real test.csv file.

### e.

In [93]:
test_df1 = pd.DataFrame(test_df)

In [94]:
Traincsv_y = train_df1.Survived
Traincsv_X = train_df1.drop(['Survived'],axis=1)
Testcsv_X = test_df1.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

In [95]:
boosted1 = GradientBoostingClassifier(n_estimators = 25, learning_rate = 0.5,
                                     max_depth = 2, random_state =0)
boosted1.fit(Traincsv_X,Traincsv_y)

GradientBoostingClassifier(learning_rate=0.5, max_depth=2, n_estimators=25,
                           random_state=0)

In [96]:
y_pred = boosted1.predict(Testcsv_X)

In [97]:
y_pred = pd.DataFrame(y_pred)

In [98]:
y_pred

,0
0,0
1,0
2,0
3,0
4,0
...,...
413,0
414,1
415,0
416,0


In [99]:
Names = test_df["PassengerId"]
testcvs_df = y_pred.join(Names)
testcvs_df.columns = ['Survived','PassengerId']

In [100]:
col_name='PassengerId'
first_col = testcvs_df.pop(col_name)
testcvs_df.insert(0, col_name, first_col)

In [101]:
testcvs_df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [102]:
testcvs_df.to_csv(index=False)
compression_opts = dict(method='zip',
                        archive_name='out.csv')  
testcvs_df.to_csv('out.zip', index=False,
          compression=compression_opts)  

In [103]:
# Kaggle ID name: JingzhiJoey
# The date submitted: 11/30/2020
# Score provided by Kaggle: 0.78229